In [49]:
import pandas as pd
import glob
import re
from tqdm import tqdm

In [50]:
data_root = '../../tuplex/test/resources/hyperspecialization/flights_all/'

In [52]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [53]:
%%time

paths = glob.glob(data_root + '*.csv.sample') + glob.glob(data_root + '*.csv')

logging.info(f'Found {len(paths)} paths to train model(s) on.')

df = pd.DataFrame()
for path in tqdm(paths):
    df = pd.concat((df, pd.read_csv(path, encoding='latin1')))

INFO:root:Found 410 paths to train model(s) on.
100%|███████████████████████████████████████████████████████████████| 410/410 [01:53<00:00,  3.60it/s]

CPU times: user 1min 9s, sys: 43.2 s, total: 1min 52s
Wall time: 1min 53s


In [43]:
# prepare data (only predict for actually delayed flights (what about diverted ones?))

In [ ]:
df.columns

In [ ]:
df['NAS_DELAY']

In [ ]:
original_df = df.copy()

In [ ]:
# get data to build model on
df = df[(df['ARR_DELAY'] >= 0.0) & ~df['NAS_DELAY'].isnull()]

In [ ]:
variables to use:
    
# quarter, month, day_of_month, day_of_week,
# carrier (op_unique_carrier), origin_airport,
# dest_airport, hour of day (arrival), hour_of day (departure),
# 

In [ ]:
df.columns

In [1]:
# XGBoost example
from sklearn.datasets import make_multilabel_classification
import numpy as np
import xgboost as xgb

In [9]:
X, y = make_multilabel_classification(
    n_samples=32, n_classes=5, n_labels=3, random_state=0
)
clf = xgb.XGBClassifier(tree_method="hist")
clf.fit(X, y)
np.testing.assert_allclose(clf.predict(X), y)

In [10]:
 cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

array([-0.49834218, -0.61491776, -0.47082612, -0.44264331, -0.32785117,
       -0.50845317, -0.33864613, -0.58610593, -0.28532898, -0.44501702,
       -0.59823456, -0.38622087, -0.36721348, -0.49661613, -0.48833209,
       -0.31357701, -0.55891194, -0.34801579, -0.51573708, -0.25158294,
       -0.43745353, -0.4149964 , -0.54938814, -0.57593607, -0.46822678,
       -0.53309795, -0.29175016, -0.41489977, -0.31530042, -0.4234729 ])

In [8]:
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
# load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
dataframe = read_csv(url, header=None)
data = dataframe.values
# split data into input and output columns
X, y = data[:, :-1], data[:, -1]
# define model
model = XGBRegressor()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean MAE: 2.113 (0.317)


In [14]:
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
image = xgb.to_graphviz(model)

#Set a different dpi (work only if format == 'png')
image.graph_attr = {'dpi':'400'}

image.render('filename', format = 'png')

'filename.png'

In [23]:
# retrieve dump
model.get_booster().get_dump()

['0:[f17<3.5] yes=1,no=2,missing=1\n\t1:[f9<1.5] yes=3,no=4,missing=3\n\t\t3:[f0<1.5] yes=7,no=8,missing=7\n\t\t\t7:leaf=-0.075000003\n\t\t\t8:leaf=0.13636364\n\t\t4:[f17<1.5] yes=9,no=10,missing=9\n\t\t\t9:[f16<0.5] yes=11,no=12,missing=11\n\t\t\t\t11:[f0<5] yes=13,no=14,missing=13\n\t\t\t\t\t13:leaf=-0.100000009\n\t\t\t\t\t14:leaf=0.075000003\n\t\t\t\t12:leaf=0.125\n\t\t\t10:leaf=-0.128571436\n\t2:[f1<3.5] yes=5,no=6,missing=5\n\t\t5:leaf=-0.128571436\n\t\t6:leaf=0.075000003\n',
 '0:[f13<4.5] yes=1,no=2,missing=1\n\t1:[f8<2.5] yes=3,no=4,missing=3\n\t\t3:[f10<5] yes=7,no=8,missing=7\n\t\t\t7:[f6<5.5] yes=13,no=14,missing=13\n\t\t\t\t13:leaf=0.139285713\n\t\t\t\t14:[f1<3] yes=15,no=16,missing=15\n\t\t\t\t\t15:leaf=0.075000003\n\t\t\t\t\t16:leaf=-0.075000003\n\t\t\t8:leaf=-0.075000003\n\t\t4:[f18<2] yes=9,no=10,missing=9\n\t\t\t9:leaf=-0.112500004\n\t\t\t10:leaf=0.075000003\n\t2:[f16<1.5] yes=5,no=6,missing=5\n\t\t5:[f0<3] yes=11,no=12,missing=11\n\t\t\t11:leaf=-0.100000009\n\t\t\t12:l

In [196]:
def string_parser(s):
    if len(re.findall(r":leaf=", s)) == 0:
        out  = re.findall(r"[\w.-]+", s)
        tabs = re.findall(r"[\t]+", s)
        if (out[4] == out[8]):
            missing_value_handling = (" or math.isnan(x['" + out[1] + "']) ")
        else:
            missing_value_handling = ""
            
        if len(tabs) > 0:
            return (re.findall(r"[\t]+", s)[0].replace('\t', '    ') + 
                    '        if state == ' + out[0] + ':\n' +
                    re.findall(r"[\t]+", s)[0].replace('\t', '    ') +
                    '            state = (' + out[4] +  
                    ' if ' +  "x['" + out[1] +"']<" + out[2] + missing_value_handling + 
                    ' else ' + out[6] + ')\n' )
        
        else:
            return ('        if state == ' + out[0] + ':\n' +
                    '            state = (' + out[4] +  
                    ' if ' +  "x['" + out[1] +"']<" + out[2] + missing_value_handling +
                    ' else ' + out[6] + ')\n' )
    else:
        out = re.findall(r"[\d.-]+", s)
        return (re.findall(r"[\t]+", s)[0].replace('\t', '    ') + 
                '        if state == ' + out[0] + ':\n    ' +
                re.findall(r"[\t]+", s)[0].replace('\t', '    ') + 
                '        return ' + out[1] + '\n')
      
def tree_parser(tree, i):
    if i == 0:
        return ('    if num_booster == 0:\n        state = 0\n'
             + "".join([string_parser(tree.split('\n')[i]) 
                        for i in range(len(tree.split('\n'))-1)]))
    else:
        return ('    elif num_booster == '+str(i)+':\n        state = 0\n'
             + "".join([string_parser(tree.split('\n')[i]) 
                        for i in range(len(tree.split('\n'))-1)])) 
    
def model_to_py(base_score, model, out_file):
    trees = model.get_dump()
    result = ["import math\n\n" 
             +"def xgb_tree(x, num_booster):\n"]
    
    for i in range(len(trees)):
        result.append(tree_parser(trees[i], i))
    
    with open(out_file, 'w') as the_file:
        the_file.write("".join(result) + "\ndef xgb_predict(x):\n    predict = " 
                       + str(base_score) + "\n"
                       + "    # initialize prediction with base score\n" 
                       + "    for i in range(" 
                       + str(len(trees))  
                       + "):\n        predict = predict + xgb_tree(x, i)"
                       + "\n    return predict")

In [36]:
train = pd.read_csv("data/titanic/train.csv")
test  = pd.read_csv("data/titanic/test.csv")
X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], 
                        label=train['Survived'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

[13:48:23] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [37]:
model_to_py(params['base_score'], model, 'xgb_model.py')

In [18]:
# use https://towardsdatascience.com/xgboost-deployment-made-easy-6e11f4b3f817 to convert to py code

In [38]:
!cat xgb_model.py

import numpy as np

def xgb_tree(x, num_booster):
    if num_booster == 0:
        state = 0
        if state == 0:
            state = (1 if x['Pclass']<3 or np.isnan(x['Pclass'])  else 2)
            if state == 1:
                state = (3 if x['Fare']<13.6458502 or np.isnan(x['Fare'])  else 4)
                if state == 3:
                    return -0.006400852
                if state == 4:
                    state = (9 if x['Age']<42.5 else 10)
                    if state == 9:
                        return 0.033544872
                    if state == 10:
                        return 0.00980747491
            if state == 2:
                state = (5 if x['Age']<6.5 else 6)
                if state == 5:
                    state = (11 if x['SibSp']<3 or np.isnan(x['SibSp'])  else 12)
                    if state == 11:
                        return 0.0521093272
                    if state == 12:
                        return -0.0224915836
                if state == 6:

In [39]:
df

NameError: name 'df' is not defined

In [59]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [62]:
# TODO: for each delay type, train an XGB boost linear regression model.
# Then: combine them all and scale them towards arr_delay

# --> figure out how this works!

# this should result in a lot of additional code.
# potentially need to add more functions/extract functionality to tuplex. -.-
# but ok, makes it better...

In [60]:
# Checking now for flights data: I.e., get a model for each delay

X_y_train = xgb.DMatrix(data=train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']], 
                        label=train['Survived', 'Fare'])
X_test    = xgb.DMatrix(data=test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']])

params = {
          'base_score': np.mean(train['Survived']),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

KeyError: ('Survived', 'Fare')

In [55]:
def xgb_tree(x, num_booster):
    if num_booster == 0:
        state = 0
        if state == 0:
            state = (1 if x['Pclass']<3 or np.isnan(x['Pclass'])  else 2)
            if state == 1:
                state = (3 if x['Fare']<13.6458502 or np.isnan(x['Fare'])  else 4)
                if state == 3:
                    return -0.006400852
                if state == 4:
                    state = (9 if x['Age']<42.5 else 10)
                    if state == 9:
                        return 0.033544872
                    if state == 10:
                        return 0.00980747491
            if state == 2:
                state = (5 if x['Age']<6.5 else 6)
                if state == 5:
                    state = (11 if x['SibSp']<3 or np.isnan(x['SibSp'])  else 12)
                    if state == 11:
                        return 0.0521093272
                    if state == 12:
                        return -0.0224915836
                if state == 6:
                    return -0.0162228346
    elif num_booster == 1:
        state = 0
        if state == 0:
            state = (1 if x['Pclass']<3 or np.isnan(x['Pclass'])  else 2)
            if state == 1:
                state = (3 if x['Fare']<52.2770996 or np.isnan(x['Fare'])  else 4)
                if state == 3:
                    state = (7 if x['Parch']<1 or np.isnan(x['Parch'])  else 8)
                    if state == 7:
                        return 0.0001473382
                    if state == 8:
                        return 0.0348955654
                if state == 4:
                    return 0.0321655758
            if state == 2:
                state = (5 if x['Age']<6.5 else 6)
                if state == 5:
                    state = (11 if x['SibSp']<3 or np.isnan(x['SibSp'])  else 12)
                    if state == 11:
                        return 0.0472049229
                    if state == 12:
                        return -0.0203923695
                if state == 6:
                    return -0.0146040637
    elif num_booster == 2:
        state = 0
        if state == 0:
            state = (1 if x['Pclass']<3 or np.isnan(x['Pclass'])  else 2)
            if state == 1:
                state = (3 if x['Fare']<13.6458502 or np.isnan(x['Fare'])  else 4)
                if state == 3:
                    return -0.00585523667
                if state == 4:
                    return 0.0201724116
            if state == 2:
                return -0.0114313215

def xgb_predict(x):
    predict = 0.3838383838383838
# initialize prediction with base score
    for i in range(3):
        predict = predict + xgb_tree(x, i)
    return predict

In [58]:
xgb_predict(train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']].iloc[0])

0.3415801640383838

In [63]:
# also edit github code to make sense! -> add more .get functions there...

In [65]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM,Unnamed: 109
0,2017,3,8,12,6,2017-08-12,B6,20409,B6,N519JB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,3,8,12,6,2017-08-12,B6,20409,B6,N570JB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,3,8,12,6,2017-08-12,B6,20409,B6,N763JB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,3,8,12,6,2017-08-12,B6,20409,B6,N763JB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,3,8,12,6,2017-08-12,B6,20409,B6,N584JB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
list(df['OP_UNIQUE_CARRIER'].unique())

['B6',
 'AS',
 'US',
 'WN',
 'OO',
 'YV',
 'DL',
 'NW',
 'CO',
 'HP',
 'UA',
 'AA',
 'TW',
 'XE',
 'MQ',
 '9E',
 'VX',
 'F9',
 'OH',
 'YX',
 'PA (1)',
 'HA',
 'EV',
 'EA',
 'NK']

In [72]:
# need unique airport list + unique carrier list
# precompute them here

In [75]:
df['ORIGIN']

0       LGA
1       SJU
2       BOS
3       SJC
4       PVD
       ... 
1995    TUS
1996    TUS
1997    TUS
1998    TUS
1999    TUS
Name: ORIGIN, Length: 820000, dtype: object

In [74]:
list(df.columns)

['YEAR',
 'QUARTER',
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'FL_DATE',
 'OP_UNIQUE_CARRIER',
 'OP_CARRIER_AIRLINE_ID',
 'OP_CARRIER',
 'TAIL_NUM',
 'OP_CARRIER_FL_NUM',
 'ORIGIN_AIRPORT_ID',
 'ORIGIN_AIRPORT_SEQ_ID',
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'ORIGIN_STATE_ABR',
 'ORIGIN_STATE_FIPS',
 'ORIGIN_STATE_NM',
 'ORIGIN_WAC',
 'DEST_AIRPORT_ID',
 'DEST_AIRPORT_SEQ_ID',
 'DEST_CITY_MARKET_ID',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_ABR',
 'DEST_STATE_FIPS',
 'DEST_STATE_NM',
 'DEST_WAC',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'DEP_DELAY_NEW',
 'DEP_DEL15',
 'DEP_DELAY_GROUP',
 'DEP_TIME_BLK',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY',
 'ARR_DELAY_NEW',
 'ARR_DEL15',
 'ARR_DELAY_GROUP',
 'ARR_TIME_BLK',
 'CANCELLED',
 'CANCELLATION_CODE',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'FLIGHTS',
 'DISTANCE',
 'DISTANCE_GROUP',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DEL

In [116]:
row = df.iloc[0]


def extract_feature_vector(row):
    
    carrier_list = [None, 'EA','UA','PI','NK','PS','AA','NW','EV','B6','HP','TW','DL','OO','F9','YV','TZ','US',
                    'MQ','OH','HA','ML (1)','XE','G4','YX','DH','AS','KH','QX','CO','FL','VX','PA (1)',
                    'WN','9E']

    airport_list = [None, 'ABE', 'ABI', 'ABQ', 'ABR', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY', 'ADK', 'ADQ', 'AEX', 'AGS', 'AKN',
                    'ALB', 'ALO', 'ALS', 'ALW', 'AMA', 'ANC', 'ANI', 'APF', 'APN', 'ART', 'ASE', 'ATL', 'ATW', 'ATY',
                    'AUS', 'AVL', 'AVP', 'AZA', 'AZO', 'BDL', 'BET', 'BFF', 'BFI', 'BFL', 'BFM', 'BGM', 'BGR', 'BHM',
                    'BIL', 'BIS', 'BJI', 'BKG', 'BLI', 'BLV', 'BMI', 'BNA', 'BOI', 'BOS', 'BPT', 'BQK', 'BQN', 'BRD',
                    'BRO', 'BRW', 'BTM', 'BTR', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CAE', 'CAK', 'CBM', 'CCR', 'CDB',
                    'CDC', 'CDV', 'CEC', 'CGI', 'CHA', 'CHO', 'CHS', 'CIC', 'CID', 'CIU', 'CKB', 'CLD', 'CLE', 'CLL',
                    'CLT', 'CMH', 'CMI', 'CMX', 'CNY', 'COD', 'COS', 'COU', 'CPR', 'CRP', 'CRW', 'CSG', 'CVG', 'CWA',
                    'CYS', 'DAB', 'DAL', 'DAY', 'DBQ', 'DCA', 'DDC', 'DEC', 'DEN', 'DET', 'DFW', 'DHN', 'DIK', 'DLG',
                    'DLH', 'DRO', 'DRT', 'DSM', 'DTW', 'DUT', 'DVL', 'EAR', 'EAT', 'EAU', 'ECP', 'EFD', 'EGE', 'EKO',
                    'ELM', 'ELP', 'ENV', 'ERI', 'ESC', 'EUG', 'EVV', 'EWN', 'EWR', 'EYW', 'FAI', 'FAR', 'FAT', 'FAY',
                    'FCA', 'FLG', 'FLL', 'FLO', 'FMN', 'FNL', 'FNT', 'FOD', 'FOE', 'FSD', 'FSM', 'FWA', 'GCC', 'GCK',
                    'GCN', 'GEG', 'GFK', 'GGG', 'GJT', 'GLH', 'GNV', 'GPT', 'GRB', 'GRI', 'GRK', 'GRR', 'GSO', 'GSP',
                    'GST', 'GTF', 'GTR', 'GUC', 'GUM', 'HDN', 'HGR', 'HHH', 'HIB', 'HKY', 'HLN', 'HNL', 'HOB', 'HOU',
                    'HPN', 'HRL', 'HSV', 'HTS', 'HVN', 'HYA', 'HYS', 'IAD', 'IAG', 'IAH', 'ICT', 'IDA', 'IFP', 'ILE',
                    'ILG', 'ILM', 'IMT', 'IND', 'INL', 'IPL', 'IPT', 'ISN', 'ISO', 'ISP', 'ITH', 'ITO', 'IYK', 'JAC', 
                    'JAN', 'JAX', 'JFK', 'JLN', 'JMS', 'JNU', 'JST', 'KOA', 'KSM', 'KTN', 'LAN', 'LAR', 'LAS', 'LAW', 
                    'LAX', 'LBB', 'LBE', 'LBF', 'LBL', 'LCH', 'LCK', 'LEX', 'LFT', 'LGA', 'LGB', 'LIH', 'LIT', 'LMT', 
                    'LNK', 'LNY', 'LRD', 'LSE', 'LWB', 'LWS', 'LYH', 'MAF', 'MAZ', 'MBS', 'MCI', 'MCN', 'MCO', 'MCW', 
                    'MDT', 'MDW', 'MEI', 'MEM', 'MFE', 'MFR', 'MGM', 'MHK', 'MHT', 'MIA', 'MIB', 'MKC', 'MKE', 'MKG',
                    'MKK', 'MLB', 'MLI', 'MLU', 'MMH', 'MOB', 'MOD', 'MOT', 'MQT', 'MRY', 'MSN', 'MSO', 'MSP', 'MSY', 
                    'MTH', 'MTJ', 'MVY', 'MWH', 'MYR', 'OAJ', 'OAK', 'OGD', 'OGG', 'OGS', 'OKC', 'OMA', 'OME', 'ONT', 
                    'ORD', 'ORF', 'ORH', 'OTH', 'OTZ', 'OWB', 'OXR', 'PAE', 'PAH', 'PBG', 'PBI', 'PDX', 'PFN', 'PGD',
                    'PGV', 'PHF', 'PHL', 'PHX', 'PIA', 'PIB', 'PIE', 'PIH', 'PIR', 'PIT', 'PLN', 'PMD', 'PNS', 'PPG', 
                    'PRC', 'PSC', 'PSE', 'PSG', 'PSM', 'PSP', 'PUB', 'PUW', 'PVD', 'PVU', 'PWM', 'RAP', 'RCA', 'RDD', 
                    'RDM', 'RDR', 'RDU', 'RFD', 'RHI', 'RIC', 'RIW', 'RKS', 'RNO', 'ROA', 'ROC', 'ROP', 'ROR', 'ROW', 
                    'RST', 'RSW', 'SAF', 'SAN', 'SAT', 'SAV', 'SBA', 'SBN', 'SBP', 'SCC', 'SCE', 'SCK', 'SDF', 'SEA', 
                    'SFB', 'SFO', 'SGF', 'SGU', 'SHD', 'SHR', 'SHV', 'SIT', 'SJC', 'SJT', 'SJU', 'SKA', 'SLC', 'SLE', 
                    'SLN', 'SMF', 'SMX', 'SNA', 'SOP', 'SPI', 'SPN', 'SPS', 'SRQ', 'STC', 'STL', 'STS', 'STT', 'STX', 
                    'SUN', 'SUX', 'SWF', 'SWO', 'SYR', 'TBN', 'TEX', 'TKI', 'TLH', 'TOL', 'TPA', 'TRI', 'TTN', 'TUL', 
                    'TUP', 'TUS', 'TVC', 'TVL', 'TWF', 'TXK', 'TYR', 'TYS', 'UCA', 'UIN', 'USA', 'UST', 'UTM', 'VCT', 
                    'VEL', 'VIS', 'VLD', 'VPS', 'WRG', 'WYS', 'XNA', 'XWA', 'YAK', 'YAP', 'YKM', 'YNG', 'YUM']

    state_list = [None, 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
                  'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
                  'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                  'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
                  'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                  'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
                  'U.S. Pacific Trust Territories and Possessions', 'U.S. Virgin Islands', 'Utah', 'Vermont',
                  'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

    # categorical variables
    quarter = row['QUARTER']
    month = row['MONTH']
    day_of_month = row['DAY_OF_MONTH']
    day_of_week = row['DAY_OF_WEEK']
    carrier = carrier_list.index(row['OP_UNIQUE_CARRIER'])
    origin_airport = airport_list.index(row['ORIGIN'])
    dest_airport = airport_list.index(row['DEST'])

    origin_state = state_list.index(row['ORIGIN_STATE_NM'])
    dest_state = state_list.index(row['DEST_STATE_NM'])

    # numerical variables
    dep_delay = row['DEP_DELAY']
    arr_delay = row['ARR_DELAY']

    crs_arr_hour = float(int(row['CRS_ARR_TIME']) // 100)
    crs_dep_hour = float(int(row['CRS_DEP_TIME']) // 100)
    crs_arr_5min = float(int(row['CRS_ARR_TIME']) % 100 // 5)
    crs_dep_5min = float(int(row['CRS_DEP_TIME']) % 100 // 5)


    features = [float(quarter), float(month), float(day_of_month),
                float(day_of_week), float(carrier), float(origin_state),
                float(dest_state), dep_delay, arr_delay,
                crs_arr_hour, crs_dep_hour,
                crs_arr_5min, crs_dep_5min]

    return features

In [117]:
# get data to build model on
train_df = df[(df['ARR_DELAY'] >= 0.0) & ~df['NAS_DELAY'].isnull()]

In [118]:
extract_feature_vector(train_df.iloc[0])

[3.0, 8.0, 12.0, 6.0, 9.0, 21.0, 5.0, 29.0, 33.0, 21.0, 17.0, 0.0, 4.0]

In [138]:
X = train_df.apply(lambda x: np.array(extract_feature_vector(x)), axis=1).values
shape = len(X), len(X[0])
X = np.concatenate(X).reshape(shape)
X

array([[ 3.,  8., 12., ..., 17.,  0.,  4.],
       [ 3.,  8., 12., ..., 21.,  6., 10.],
       [ 3.,  8., 12., ..., 11.,  2.,  4.],
       ...,
       [ 1.,  3., 31., ..., 16.,  6.,  3.],
       [ 1.,  3., 31., ..., 18.,  1.,  7.],
       [ 1.,  3., 31., ..., 19.,  2., 11.]])

In [141]:
y = train_df['NAS_DELAY'].values
y.shape, X.shape

((82176,), (82176, 13))

In [185]:
from sklearn.metrics import mean_absolute_error


base_score = np.mean(y)
model = XGBRegressor(objective ='reg:squarederror',
             tree_method='hist',
             base_score=base_score,
             eval_metric=mean_absolute_error,
            max_depth=3,
            eta=0.8,
            gamma=3,
            n_estimators=100)

In [197]:
model.fit(X, y, eval_set=[(X, y)])

[0]	validation_0-rmse:25.75855	validation_0-mean_absolute_error:13.20058
[1]	validation_0-rmse:25.16909	validation_0-mean_absolute_error:12.21410
[2]	validation_0-rmse:24.85085	validation_0-mean_absolute_error:11.27254
[3]	validation_0-rmse:24.53439	validation_0-mean_absolute_error:10.93720
[4]	validation_0-rmse:24.23343	validation_0-mean_absolute_error:10.81144
[5]	validation_0-rmse:23.90490	validation_0-mean_absolute_error:10.65180
[6]	validation_0-rmse:23.58432	validation_0-mean_absolute_error:10.61168
[7]	validation_0-rmse:23.25871	validation_0-mean_absolute_error:10.37150
[8]	validation_0-rmse:23.18967	validation_0-mean_absolute_error:10.34685
[9]	validation_0-rmse:23.07079	validation_0-mean_absolute_error:10.30804
[10]	validation_0-rmse:23.02160	validation_0-mean_absolute_error:10.29095
[11]	validation_0-rmse:22.93044	validation_0-mean_absolute_error:10.13800
[12]	validation_0-rmse:22.81470	validation_0-mean_absolute_error:9.82029
[13]	validation_0-rmse:22.73855	validation_0-mean

XGBRegressor(base_score=14.015040887850468, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.8,
             eval_metric=<function mean_absolute_error at 0x12aa4e440>,
             feature_types=None, gamma=3, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [198]:
model_to_py(base_score, model.get_booster(), 'xgb_model.py')

In [199]:
model.predict([X[0,:]])

array([7.4927807], dtype=float32)

In [200]:
model.predict(X)

array([ 7.4927807, -1.5349699, 22.772686 , ...,  1.3999364,  2.5768185,
        1.7052329], dtype=float32)

In [201]:
X[0,:].shape

(13,)

In [203]:
# check custom prediction
from xgb_model import *

In [207]:
# per default zipped with 'f...'

# replace x'f\d+'' with the respective number!
def model_predict(x):
    
    d = {}
    for i in range(13):
        d[f'f{i}'] = x[i]
    return xgb_predict(d)

In [208]:
model_predict(X[0,:])

7.492778305590468

In [135]:
# xgb boost regressor

params = {
          'base_score': np.mean(y),
          'eta':  0.1,
          'max_depth': 3,
          'gamma' :3,
          'objective'   :'reg:linear',
          'eval_metric' :'mae'
         }
model = xgb.train(params=params, 
                  dtrain=X_y_train, 
                  num_boost_round=3)

array([[ 3.,  8., 12., ..., 17.,  0.,  4.],
       [ 3.,  8., 12., ..., 21.,  6., 10.],
       [ 3.,  8., 12., ..., 11.,  2.,  4.],
       ...,
       [ 1.,  3., 31., ..., 16.,  6.,  3.],
       [ 1.,  3., 31., ..., 18.,  1.,  7.],
       [ 1.,  3., 31., ..., 19.,  2., 11.]])

#### getting list via spark
```
val df = spark.read.format("csv").option("header","true").load("/hot/data/flights_all/*.csv")
var res = df.select("ORIGIN").distinct().collect()
print("'" + res.map(arr => arr(0)).mkString("','") + "'")
```

In [88]:
airport_list = ['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY', 'ADK', 'ADQ', 'AEX', 'AGS', 'AKN',
                'ALB', 'ALO', 'ALS', 'ALW', 'AMA', 'ANC', 'ANI', 'APF', 'APN', 'ART', 'ASE', 'ATL', 'ATW', 'ATY',
                'AUS', 'AVL', 'AVP', 'AZA', 'AZO', 'BDL', 'BET', 'BFF', 'BFI', 'BFL', 'BFM', 'BGM', 'BGR', 'BHM',
                'BIL', 'BIS', 'BJI', 'BKG', 'BLI', 'BLV', 'BMI', 'BNA', 'BOI', 'BOS', 'BPT', 'BQK', 'BQN', 'BRD',
                'BRO', 'BRW', 'BTM', 'BTR', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CAE', 'CAK', 'CBM', 'CCR', 'CDB',
                'CDC', 'CDV', 'CEC', 'CGI', 'CHA', 'CHO', 'CHS', 'CIC', 'CID', 'CIU', 'CKB', 'CLD', 'CLE', 'CLL',
                'CLT', 'CMH', 'CMI', 'CMX', 'CNY', 'COD', 'COS', 'COU', 'CPR', 'CRP', 'CRW', 'CSG', 'CVG', 'CWA',
                'CYS', 'DAB', 'DAL', 'DAY', 'DBQ', 'DCA', 'DDC', 'DEC', 'DEN', 'DET', 'DFW', 'DHN', 'DIK', 'DLG',
                'DLH', 'DRO', 'DRT', 'DSM', 'DTW', 'DUT', 'DVL', 'EAR', 'EAT', 'EAU', 'ECP', 'EFD', 'EGE', 'EKO',
                'ELM', 'ELP', 'ENV', 'ERI', 'ESC', 'EUG', 'EVV', 'EWN', 'EWR', 'EYW', 'FAI', 'FAR', 'FAT', 'FAY',
                'FCA', 'FLG', 'FLL', 'FLO', 'FMN', 'FNL', 'FNT', 'FOD', 'FOE', 'FSD', 'FSM', 'FWA', 'GCC', 'GCK',
                'GCN', 'GEG', 'GFK', 'GGG', 'GJT', 'GLH', 'GNV', 'GPT', 'GRB', 'GRI', 'GRK', 'GRR', 'GSO', 'GSP',
                'GST', 'GTF', 'GTR', 'GUC', 'GUM', 'HDN', 'HGR', 'HHH', 'HIB', 'HKY', 'HLN', 'HNL', 'HOB', 'HOU',
                'HPN', 'HRL', 'HSV', 'HTS', 'HVN', 'HYA', 'HYS', 'IAD', 'IAG', 'IAH', 'ICT', 'IDA', 'IFP', 'ILE',
                'ILG', 'ILM', 'IMT', 'IND', 'INL', 'IPL', 'IPT', 'ISN', 'ISO', 'ISP', 'ITH', 'ITO', 'IYK', 'JAC', 
                'JAN', 'JAX', 'JFK', 'JLN', 'JMS', 'JNU', 'JST', 'KOA', 'KSM', 'KTN', 'LAN', 'LAR', 'LAS', 'LAW', 
                'LAX', 'LBB', 'LBE', 'LBF', 'LBL', 'LCH', 'LCK', 'LEX', 'LFT', 'LGA', 'LGB', 'LIH', 'LIT', 'LMT', 
                'LNK', 'LNY', 'LRD', 'LSE', 'LWB', 'LWS', 'LYH', 'MAF', 'MAZ', 'MBS', 'MCI', 'MCN', 'MCO', 'MCW', 
                'MDT', 'MDW', 'MEI', 'MEM', 'MFE', 'MFR', 'MGM', 'MHK', 'MHT', 'MIA', 'MIB', 'MKC', 'MKE', 'MKG',
                'MKK', 'MLB', 'MLI', 'MLU', 'MMH', 'MOB', 'MOD', 'MOT', 'MQT', 'MRY', 'MSN', 'MSO', 'MSP', 'MSY', 
                'MTH', 'MTJ', 'MVY', 'MWH', 'MYR', 'OAJ', 'OAK', 'OGD', 'OGG', 'OGS', 'OKC', 'OMA', 'OME', 'ONT', 
                'ORD', 'ORF', 'ORH', 'OTH', 'OTZ', 'OWB', 'OXR', 'PAE', 'PAH', 'PBG', 'PBI', 'PDX', 'PFN', 'PGD',
                'PGV', 'PHF', 'PHL', 'PHX', 'PIA', 'PIB', 'PIE', 'PIH', 'PIR', 'PIT', 'PLN', 'PMD', 'PNS', 'PPG', 
                'PRC', 'PSC', 'PSE', 'PSG', 'PSM', 'PSP', 'PUB', 'PUW', 'PVD', 'PVU', 'PWM', 'RAP', 'RCA', 'RDD', 
                'RDM', 'RDR', 'RDU', 'RFD', 'RHI', 'RIC', 'RIW', 'RKS', 'RNO', 'ROA', 'ROC', 'ROP', 'ROR', 'ROW', 
                'RST', 'RSW', 'SAF', 'SAN', 'SAT', 'SAV', 'SBA', 'SBN', 'SBP', 'SCC', 'SCE', 'SCK', 'SDF', 'SEA', 
                'SFB', 'SFO', 'SGF', 'SGU', 'SHD', 'SHR', 'SHV', 'SIT', 'SJC', 'SJT', 'SJU', 'SKA', 'SLC', 'SLE', 
                'SLN', 'SMF', 'SMX', 'SNA', 'SOP', 'SPI', 'SPN', 'SPS', 'SRQ', 'STC', 'STL', 'STS', 'STT', 'STX', 
                'SUN', 'SUX', 'SWF', 'SWO', 'SYR', 'TBN', 'TEX', 'TKI', 'TLH', 'TOL', 'TPA', 'TRI', 'TTN', 'TUL', 
                'TUP', 'TUS', 'TVC', 'TVL', 'TWF', 'TXK', 'TYR', 'TYS', 'UCA', 'UIN', 'USA', 'UST', 'UTM', 'VCT', 
                'VEL', 'VIS', 'VLD', 'VPS', 'WRG', 'WYS', 'XNA', 'XWA', 'YAK', 'YAP', 'YKM', 'YNG', 'YUM']

In [76]:
dest_list = ['BGM','PSE','DLG','INL','MSY','PPG','GEG','DRT','SNA','BUR','GRB','GTF','FOD','IFP','IDA','GRR','LWB','PVU','JLN','EUG','PSG','ATY','PVD','GSO','MYR','ISO','OAK','EAR','MSN','FAR','BTM','COD','FSM','MQT','SCC','RCA','DET','ESC','FNL','DCA','RFD','MLU','CID','ROP','GTR','LWS','PIB','SWO','UTM','WRG','MTH','HLN','LNY','CIU','IAG','DDC','LEX','RDM','FLO','JMS','ORF','SCE','EVV','KTN','CWA','CRW','OGS','VCT','SAV','GCK','TRI','CDV','CMH','MOD','SPN','YNG','LBF','MWH','CAK','UCA','TYR','ADK','CDB','CHO','MOB','PNS','MCN','DIK','CEC','LIH','IAH','FMN','HNL','SLN','SHV','ERI','GST','SJC','CVG','TOL','LGA','BUF','TLH','CDC','ACT','HPN','RDD','TBN','AUS','MLI','GCC','MIB','SJU','ATW','DHN','LYH','AVL','LGB','GJT','MAZ','GFK','BFL','DUT','CNY','RIW','SRQ','RNO','EYW','SBN','BJI','TTN','RST','JAC','CHS','HGR','RSW','HRL','TUL','IPL','AMA','ISP','MKC','BOS','PUW','MLB','MAF','TUP','EWR','LAS','BIS','OGD','JAN','FAI','ITO','IMT','UIN','XNA','HHH','DLH','MKK','HYA','DEN','RHI','EWN','SGU','ALS','ALB','CPR','LNK','OME','GRI','IAD','PSP','SBA','BOI','IYK','MEI','LAR','HOB','DRO','BRO','BRD','BMI','RKS','SEA','CMI','LAN','VEL','LRD','PBG','VLD','HYS','PSM','MCI','BFI','PIR','FLG','GRK','BNA','CLT','TVC','CLL','UST','CGI','PAE','PSC','BLI','CIC','ORH','PBI','PIE','ABQ','SDF','ART','ACV','SCK','LAW','SLE','BDL','MRY','DAL','ITH','DBQ','USA','CLE','TVL','APF','PDX','ACK','MIA','MFR','ILG','TWF','TPA','BWI','BKG','CMX','APN','PRC','OKC','ROA','SMF','YAP','SPI','BRW','RDR','OTH','SFB','ABI','OXR','BFF','MBS','ELM','PHX','FCA','ABR','DVL','STL','PWM','BET','ABY','DFW','ABE','MHT','TXK','VIS','GSP','LSE','MMH','BFM','FAY','STX','HDN','EFD','LMT','GUC','LBB','EKO','CBM','CRP','EAT','ANI','EGE','HVN','FSD','SWF','CSG','SUN','BQK','SFO','MEM','SAF','ELP','GCN','BHM','FLL','ATL','FNT','YKM','PIH','AZA','DEC','MCW','RIC','AKN','LCK','DAY','PHF','OMA','SJT','LCH','TEX','STC','VPS','BPT','MHK','KSM','LIT','MVY','ICT','CAE','FAT','ECP','PFN','SOP','ORD','AVP','LBE','LBL','BTV','MKG','COU','IPT','BIL','SPS','AEX','ILM','PUB','SMX','PIA','GUM','RDU','BQN','MFE','HIB','PGV','MKE','SKA','XWA','SYR','CCR','ISN','BLV','HSV','LFT','PIT','TUS','ACY','MTJ','ROW','AZO','MDW','PLN','COS','OAJ','CKB','JNU','IND','ALO','EAU','KOA','GPT','MGM','JST','ALW','OWB','DTW','HOU','CHA','TYS','YUM','ADQ','ONT','FWA','MDT','JAX','STS','MSP','LAX','MOT','HTS','BTR','BGR','SIT','MCO','ROC','AGS','SGF','ROR','OTZ','WYS','SAN','BZN','GGG','SHD','YAK','DAB','JFK','SUX','ANC','PAH','MSO','GNV','ILE','CYS','PHL','OGG','PGD','DSM','FOE','SLC','SAT','PMD','GLH','SHR','STT','RAP','SBP','CLD','HKY','ASE']

In [83]:
len(airport_list), len(dest_list)

(433, 431)

In [87]:
airport_list = list(set(airport_list) | set(dest_list))

'[' + ', '.join(sorted(list(map(lambda x: '\'{}\''.format(x), airport_list)))) + ']'

"['ABE', 'ABI', 'ABQ', 'ABR', 'ABY', 'ACK', 'ACT', 'ACV', 'ACY', 'ADK', 'ADQ', 'AEX', 'AGS', 'AKN', 'ALB', 'ALO', 'ALS', 'ALW', 'AMA', 'ANC', 'ANI', 'APF', 'APN', 'ART', 'ASE', 'ATL', 'ATW', 'ATY', 'AUS', 'AVL', 'AVP', 'AZA', 'AZO', 'BDL', 'BET', 'BFF', 'BFI', 'BFL', 'BFM', 'BGM', 'BGR', 'BHM', 'BIL', 'BIS', 'BJI', 'BKG', 'BLI', 'BLV', 'BMI', 'BNA', 'BOI', 'BOS', 'BPT', 'BQK', 'BQN', 'BRD', 'BRO', 'BRW', 'BTM', 'BTR', 'BTV', 'BUF', 'BUR', 'BWI', 'BZN', 'CAE', 'CAK', 'CBM', 'CCR', 'CDB', 'CDC', 'CDV', 'CEC', 'CGI', 'CHA', 'CHO', 'CHS', 'CIC', 'CID', 'CIU', 'CKB', 'CLD', 'CLE', 'CLL', 'CLT', 'CMH', 'CMI', 'CMX', 'CNY', 'COD', 'COS', 'COU', 'CPR', 'CRP', 'CRW', 'CSG', 'CVG', 'CWA', 'CYS', 'DAB', 'DAL', 'DAY', 'DBQ', 'DCA', 'DDC', 'DEC', 'DEN', 'DET', 'DFW', 'DHN', 'DIK', 'DLG', 'DLH', 'DRO', 'DRT', 'DSM', 'DTW', 'DUT', 'DVL', 'EAR', 'EAT', 'EAU', 'ECP', 'EFD', 'EGE', 'EKO', 'ELM', 'ELP', 'ENV', 'ERI', 'ESC', 'EUG', 'EVV', 'EWN', 'EWR', 'EYW', 'FAI', 'FAR', 'FAT', 'FAY', 'FCA', 'FLG', 'FLL

In [94]:
set(['Utah','Hawaii','U.S. Virgin Islands','Minnesota','U.S. Pacific Trust Territories and Possessions','Ohio','Arkansas','Oregon','Texas','null','North Dakota','Pennsylvania','Connecticut','Nebraska','Vermont','Nevada','Puerto Rico','Washington','Illinois','Oklahoma','Delaware','Alaska','New Mexico','West Virginia','Missouri','Rhode Island','Georgia','Montana','Michigan','Virginia','North Carolina','Wyoming','Kansas','New Jersey','Maryland','Alabama','Arizona','Iowa','Massachusetts','Kentucky','Louisiana','Mississippi','Tennessee','New Hampshire','Indiana','Florida','Idaho','South Carolina','South Dakota','California','New York','Wisconsin','Colorado','Maine'] + ['Utah','Hawaii','U.S. Virgin Islands','Minnesota','U.S. Pacific Trust Territories and Possessions','Ohio','Arkansas','Oregon','Texas','null','North Dakota','Pennsylvania','Connecticut','Nebraska','Vermont','Nevada','Puerto Rico','Washington','Illinois','Oklahoma','Delaware','Alaska','New Mexico','West Virginia','Missouri','Rhode Island','Georgia','Montana','Michigan','Virginia','North Carolina','Wyoming','Kansas','New Jersey','Maryland','Alabama','Arizona','Iowa','Massachusetts','Kentucky','Louisiana','Mississippi','Tennessee','New Hampshire','Florida','Indiana','Idaho','South Carolina','South Dakota','California','New York','Wisconsin','Colorado','Maine'])

{'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'U.S. Pacific Trust Territories and Possessions',
 'U.S. Virgin Islands',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',
 'null'}

In [98]:
state_list = list({'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'U.S. Pacific Trust Territories and Possessions',
 'U.S. Virgin Islands',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming',})

'[' + ', '.join(sorted(list(map(lambda x: '\'{}\''.format(x), state_list)))) + ']'

"['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'U.S. Pacific Trust Territories and Possessions', 'U.S. Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']"

In [ ]:
# quarter, month, day_of_month, day_of_week,
# carrier (op_unique_carrier), origin_airport,
# dest_airport, hour of day (arrival), hour_of day (departure),